In [1]:
import pandas as pd
import numpy as np

import math

# datascraping for state - state code mapper
import requests # for datascrape
from bs4 import BeautifulSoup # datascrape

# distance calculation
import geopy.distance

# to time code
from timeit import default_timer as timer
import sys

# data imputation with MissForest
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest # data imputation

# check if file exists
from pathlib import Path

In [2]:
# Load accident dataset
accident_data_path = '/Users/jonathanwozny/Documents/GraduateSchool/Fall2022/COP5725/Project/US_Accidents_Dec21_updated.csv'
df_acc = pd.read_csv(accident_data_path)

# Load Airport dataset
airport_data_path = '/Users/jonathanwozny/Documents/GraduateSchool/Fall2022/COP5725/Project/us-airports.csv'
df_air = pd.read_csv(airport_data_path)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.max_rows', None)
print(df_air.isna().sum())
pd.set_option('display.max_rows', 10)

id                       0
ident                    0
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft          2006
continent            29443
country_name             0
iso_country              0
region_name              0
iso_region               0
local_region             0
municipality            62
scheduled_service        0
gps_code              8863
iata_code            27535
local_code            7972
home_link            28483
wikipedia_link       27063
keywords             24478
score                    0
last_updated             0
dtype: int64


In [5]:
pd.set_option('display.max_rows', None)
print(df_acc.isna().sum())
pd.set_option('display.max_rows', 10)

ID                             0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                        0
End_Lng                        0
Distance(mi)                   0
Description                    0
Number                   1743911
Street                         2
Side                           0
City                         137
County                         0
State                          0
Zipcode                     1319
Country                        0
Timezone                    3659
Airport_Code                9549
Weather_Timestamp          50736
Temperature(F)             69274
Wind_Chill(F)             469643
Humidity(%)                73092
Pressure(in)               59200
Visibility(mi)             70546
Wind_Direction             73775
Wind_Speed(mph)           157944
Precipitation(in)         549458
Weather_Condition          70636
Amenity   

In [6]:
df_air.loc[df_air['gps_code'].isna(), 'gps_code'] = df_air.loc[df_air['gps_code'].isna(), 'ident']

In [7]:
# combine datasets - this is useful for when city names don't match exactly...can just choose one... 
#    i.e. St Louis != St. Louis
df = df_acc.merge(df_air, left_on=['Airport_Code'], right_on = ['ident'], how='left')

In [8]:
# Environment(Env_ID : integer, Precipitation : decimal, Visibility : integer, 
#             Temperature : decimal, Humidity : decimal, weather_condition : string, 
#             Light_Or_Darkness : boolean, Air_pressure : decimal, Wind_Speed : decimal, Wind_Direction : string) 

keep_cols = ['ID', 'Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Temperature(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'ident', 'type', 'name', 'latitude_deg',
       'longitude_deg', 'region_name',  'local_region',
       'municipality', 'gps_code']
df = df[keep_cols]

In [9]:
df.loc[df["City"].isna(), "City"] = df.loc[df["City"].isna(), "municipality"]

In [10]:
pd.set_option('display.max_rows', 10)
df

,ID,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,ident,type,name,latitude_deg,longitude_deg,region_name,local_region,municipality,gps_code
0,A-1,3,2016-02-08 00:37:08,40.108910,-83.092860,3.230,Outerbelt E,R,Dublin,Franklin,OH,43017,US,US/Eastern,KOSU,42.1,58.0,29.76,10.0,SW,10.4,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,KOSU,medium_airport,The Ohio State University Airport - Don Scott ...,40.0798,-83.072998,Ohio,OH,Columbus,KOSU
1,A-2,2,2016-02-08 05:56:20,39.865420,-84.062800,0.747,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,36.9,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,KFFO,medium_airport,Wright-Patterson Air Force Base,39.826099,-84.048302,Ohio,OH,Dayton,KFFO
2,A-3,2,2016-02-08 06:15:39,39.102660,-84.524680,0.055,I-75 S,R,Cincinnati,Hamilton,OH,45203,US,US/Eastern,KLUK,36.0,97.0,29.70,10.0,Calm,NaN,0.02,Overcast,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,KLUK,medium_airport,Cincinnati Municipal Airport Lunken Field,39.10329819,-84.41860199,Ohio,OH,Cincinnati,KLUK
3,A-4,2,2016-02-08 06:51:45,41.062130,-81.537840,0.123,I-77 N,R,Akron,Summit,OH,44311,US,US/Eastern,KAKR,39.0,55.0,29.65,10.0,Calm,NaN,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,KAKR,medium_airport,Akron Executive Airport,41.037498,-81.466904,Ohio,OH,Akron,KAKR
4,A-5,3,2016-02-08 07:53:43,39.172393,-84.492792,0.500,I-75 S,R,Cincinnati,Hamilton,OH,45217,US,US/Eastern,KLUK,37.0,93.0,29.69,10.0,WSW,10.4,0.01,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,KLUK,medium_airport,Cincinnati Municipal Airport Lunken Field,39.10329819,-84.41860199,Ohio,OH,Cincinnati,KLUK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,A-2845338,2,2019-08-23 18:03:25,34.002480,-117.379360,0.543,Pomona Fwy E,R,Riverside,Riverside,CA,92501,US,US/Pacific,KRAL,86.0,40.0,28.92,10.0,W,13.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,KRAL,medium_airport,Riverside Municipal Airport,33.9519,-117.445,California,CA,Riverside,KRAL
2845338,A-2845339,2,2019-08-23 19:11:30,32.766960,-117.148060,0.338,I-8 W,R,San Diego,San Diego,CA,92108,US,US/Pacific,KMYF,70.0,73.0,29.39,10.0,SW,6.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,KMYF,small_airport,Montgomery-Gibbs Executive Airport,32.8157,-117.139999,California,CA,San Diego,KMYF
2845339,A-2845340,2,2019-08-23 19:00:21,33.775450,-117.847790,0.561,Garden Grove Fwy,R,Orange,Orange,CA,92866,US,US/Pacific,KSNA,73.0,64.0,29.74,10.0,SSW,10.0,0.00,Partly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Day,KSNA,large_airport,John Wayne Airport-Orange County Airport,33.675701,-117.867996,California,CA,Santa Ana,KSNA
2845340,A-2845341,2,2019-08-23 19:00:21,33.992460,-118.403020,0.772,San Diego Fwy S,R,Culver City,Los Angeles,CA,90230,US,US/Pacific,KSMO,71.0,81.0,29.62,10.0,SW,8.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,KSMO,medium_airport,Santa Monica Municipal Airport,34.0158,-118.450996,California,CA,Santa Monica,KSMO


# Handle NaN Values

In [11]:
pd.set_option('display.max_rows', None)
print(df.isna().sum())
pd.set_option('display.max_rows', 10)

ID                        0
Severity                  0
Start_Time                0
Start_Lat                 0
Start_Lng                 0
Distance(mi)              0
Street                    2
Side                      0
City                     56
County                    0
State                     0
Zipcode                1319
Country                   0
Timezone               3659
Airport_Code           9549
Temperature(F)        69274
Humidity(%)           73092
Pressure(in)          59200
Visibility(mi)        70546
Wind_Direction        73775
Wind_Speed(mph)      157944
Precipitation(in)    549458
Weather_Condition     70636
Amenity                   0
Bump                      0
Crossing                  0
Give_Way                  0
Junction                  0
No_Exit                   0
Railway                   0
Roundabout                0
Station                   0
Stop                      0
Traffic_Calming           0
Traffic_Signal            0
Turning_Loop        

## Fill Missing Airport Values (after merging Accident and Airport dataset)

In [12]:
air_cols_toRet = ['type', 'name', 'latitude_deg',
       'longitude_deg', 'region_name',  'local_region',
       'municipality']

In [13]:
# remove NaN values to make function work
df = df[df['City'].notna()]
df.reset_index(inplace=True)
df.drop('index', axis = 1, inplace=True)


df_air = df_air[df_air['municipality'].notna()]
df_air.reset_index(inplace=True)
df_air.drop('index', axis = 1, inplace=True)

# check that values are dropped
print(df['City'].isna().sum())
print(df_air['municipality'].isna().sum())

/Users/jonathanwozny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


0
0


#### For each row of the dataframe (using apply and lambda function), this function finds the nearest airport to the accident based on the latitude and longitude coordinates

In [14]:
# speed up to use with apply and lambda function for dataframe
# ASSUMES NO NAN VALUES FOR City, State, Municipality, local_region, Start_Lat, Start_Lng, latitude_deg, longitude_deg
def NearestAirportSearchDF(dfRow, air_cols = air_cols_toRet, df_air=df_air):
    start = timer()
    closest_index = 'None'
    closest_ind = -1
    closest_dist = float(sys.maxsize)
    coords_1 = [dfRow['Start_Lat'], dfRow['Start_Lng']]
    retList = []
    # search to find city first, and if not, search through state
    temp_df = df_air.loc[(df_air['local_region'] == dfRow['State']) & (dfRow['City'] == df_air['municipality'])]
    if (len(temp_df.index)):
        for index, row in temp_df.iterrows():
            coords_2 = [row['latitude_deg'], row['longitude_deg']]
            dist = geopy.distance.geodesic(coords_1, coords_2).mi
            if (dist < closest_dist):
                closest_ind = index
                closest_index = row['ident']
                closest_dist = dist
    else:
        for index, row in df_air.loc[df_air['local_region'] == dfRow['State']].iterrows():
            coords_2 = [row['latitude_deg'], row['longitude_deg']]
            dist = geopy.distance.geodesic(coords_1, coords_2).mi
            if (dist < closest_dist):
                closest_ind = index
                closest_index = row['ident']
                closest_dist = dist
    dfRow['ident'] = closest_index
    dfRow['Nearest_Air_Dist'] = closest_dist
    dfRow[air_cols_toRet] = df_air.iloc[closest_ind][air_cols_toRet]
    end = timer()
    print(end-start)
    return dfRow

In [15]:
# check if the file already exists because it may take long depending on how many values
my_file = Path("Acc_Air_Merge.csv")

if not my_file.is_file():
    ### NOTE: THIS WILL TAKE ABOUT 7-8 HOURS FOR THE ACCIDENT DATASET
    df['Nearest_Air_Dist'] = 0
    df.loc[df['ident'].isna()] = df.loc[df['ident'].isna()].apply(lambda x: NearestAirportSearchDF(x), axis=1)
    df.to_csv('Acc_Air_Merge.csv')
else:
    df = pd.read_csv(my_file)

In [16]:
# confirm no null values for desired airport columns
print(df[['ident'] + air_cols_toRet].isna().sum())

ident            0
type             0
name             0
latitude_deg     0
longitude_deg    0
region_name      0
local_region     0
municipality     0
dtype: int64


#### This Function Calculates the Nearest Distance to the Airport, Given Nearest Airport Already Found
##### Basically the above function to find the nearest airport, did not record some of the distances (might might have been because Nearest_Air_Dist wasn't created outside of the function

In [17]:
def NearestAirDistCalc(row):
    coords_1 = (row['Start_Lat'], row['Start_Lng'])
    coords_2 = (row['latitude_deg'], row['longitude_deg'])
    dist = geopy.distance.geodesic(coords_1, coords_2).mi
    row['Nearest_Air_Dist'] = dist
    return row

In [18]:
runFlag = False
my_file = Path("Acc_Air_Merge.csv")

if (not my_file.is_file()) | runFlag:
    if (df['Nearest_Air_Dist'].iloc[0] == 0):
        df = df.apply(lambda x: NearestAirDistCalc(x), axis=1)
        df.to_csv('Acc_Air_Merge.csv')

## Environment NaN's
- How to deal with NaN values for Environment:
    - Precipitation     - set to 0
    - Visibility        - impute
    - Temperature       - Average of City, State or Average of State if City,State does not exist
    - Humidity          - Average of City, State or Average of State if City,State does not exist
    - Pressure          - Average of City, State or Average of State if City,State does not exist
    - Weather_Condition - if precipitation is equal to 0 then clear...else, value of the next value for the same state 
    - Light_or_Darkness - check time of day before 7 pm and after 6 am is light
    - Wind_Speed        - value of the next value for the same state after, set rest of values to 0
    - Wind_Direction    - if wind_speed is equal to 0 or NaN then calm...else set rest of values to calm

#### This function finds the average value for the corresponding city, state (or just state) combination for the desired column(s)

In [19]:
def AverageOfCityState(val, col, cs_avg, s_avg):
    # if it is nan, then do state groupby, otherwise do citystate groupby to get avg
    if (math.isnan(float(cs_avg.loc[val['City']].loc[val['State']][col]))):
        val[col] = s_avg.loc[val['State']][col]
    else:
        val[col] = cs_avg.loc[val['City']].loc[val['State']][col]
    
    return val

#### This function fills the cols with the strategy/function above

In [20]:
def FillWithCityStateAvg(data, fill_cols):
    print("FillWithCityStateAvg")
    
    cityStateCols = ['City', 'State']
    # groupby dataframes
    citystate_avg = data[cityStateCols + fill_cols].groupby(cityStateCols).mean()
    state_avg = data[['State'] + fill_cols].groupby('State').mean()

    # fill the columns with averages of city and state or average of just state
    for col in fill_cols:
        data.loc[data[col].isna(), cityStateCols + [col]] = data.loc[data[col].isna(), cityStateCols + [col]].apply(lambda x: AverageOfCityState(x, col, citystate_avg, state_avg), axis = 1)
    
    return data[fill_cols]
        

#### This function imputes values for given column(s) using given reference columns

In [21]:
def ImputeValues(data, imp_col, ref_cols, make_cat = True):
    print("Impute Values")
    
    # setup for imputation
    fakeFill = 9999999
    data[imp_col].fillna(fakeFill, inplace=True)
    temp_imp = data[[imp_col] + ref_cols]
    
    # encode non-integer values
    for col in temp_imp.columns:
        if (temp_imp[col].dtype == 'O'):
            # put all unique values into list
            myList = temp_imp[col].unique()
            # dictionary to map values to encoded integers by index
            mapper = dict(map(reversed,enumerate(myList))) 
            # map to encoded values
            temp_imp[col] = temp_imp[col].apply(lambda x: mapper[x])
    
    # impute with MissForest
    imputer = MissForest(missing_values = fakeFill, criterion=('squared_error', 'gini')) # instantiate and replace set value
    X_imputed = imputer.fit_transform(temp_imp)
    X_imputedDF = pd.DataFrame(X_imputed, columns = [imp_col] + ref_cols)
    if make_cat:
        X_imputedDF[imp_col] = X_imputedDF[imp_col].apply(lambda x: round(x))

    return X_imputedDF[imp_col]


#### This function fills all the Environment columns based on the strategy in the higher level header above

In [22]:
def FillEnvironmentNaN(data):
    # Precipitation
    data['Precipitation(in)'].fillna(0, inplace=True)
    
    # Light_or_Darkness
    data['Light_Or_Darkness'] = data['Start_Time'].apply(lambda x: 'Day' if (int(x[11:13]) < 19  and int(x[11:13]) > 6) else 'Night')
    
    # Weather_Condition - if precipitation is equal to 0 then clear...if equal to value, then set to rain
    # re-encode to 4 categories
    data.loc[(data['Weather_Condition'].isna()) & (data['Precipitation(in)'] == 0), 'Weather_Condition'] = 'Clear'
    data.loc[(data['Weather_Condition'].isna()) & (data['Precipitation(in)'] != 0), 'Weather_Condition'] = 'Rain'
    
    new_wcond = {'Snow' : ['Snow', 'Ice', 'Hail', 'Wintry', 'Sleet'], 
             'Clear' : ['Overcast', 'Cloudy', 'Clear', 'Fair', 'Scattered Clouds'], 
             'Rain' : ['Rain', 'Drizzle', 'Showers', 'Thunder', 'T-Storm', 'Showers', 'Precipitation'], 
             'Not_Clear': ['Fog', 'Smoke', 'Sand', 'Mist', 'Dust', 'Haze','Squalls', 'Funnel', 'Tornado', 'Volcanic Ash']}
    for key in new_wcond:
        df.loc[df['Weather_Condition'].str.contains('|'.join(new_wcond[key])) == True, 'Weather_Condition'] = key
    
    # Wind_Speed - if Wind_Direction has value, but Wind_Speed is NaN, set to 0 - now it has about the same amount of NaN's as Wind_Direction
    data.loc[(df_acc['Wind_Speed(mph)'].isna()) & (data['Wind_Direction'].notna()), 'Wind_Speed(mph)'] = 0
    
    # Wind_Direction - if Wind_Speed is Nan, then calm
    data.loc[(data['Wind_Speed(mph)'].isna()) & (data['Wind_Direction'].notna()), 'Wind_Direction'] = 'Calm'
    
    # Leftover Wind_Speed and Wind_Direction will be set to calm and 0
    data[['Wind_Speed(mph)', 'Wind_Direction']] = [0, 'Calm']
    
    # Fill values with averages of the corresponding city and state or the corresponding state
    avg_fill_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)']
    data[avg_fill_cols] = FillWithCityStateAvg(data, avg_fill_cols)
    
    
    # Impute Visibility
    imp_col = 'Visibility(mi)'
    ref_cols = ['Precipitation(in)', 'Wind_Speed(mph)', 'Weather_Condition']
    data[imp_col] = ImputeValues(data, imp_col, ref_cols)
    data.loc[data[imp_col] > 10, imp_col] = 10
    
    return data


In [23]:
# handle environment NaN's
# columns that cannot have null values
    # City, State
df = FillEnvironmentNaN(df)

FillWithCityStateAvg
Impute Values


<ipython-input-21-718cdbb4230a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_imp[col] = temp_imp[col].apply(lambda x: mapper[x])


Iteration: 0
Iteration: 1
Iteration: 2


In [25]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [26]:
pd.set_option('display.max_rows', None)
print(df.isna().sum())
pd.set_option('display.max_rows', 10)

ID                        0
Severity                  0
Start_Time                0
Start_Lat                 0
Start_Lng                 0
Distance(mi)              0
Street                    2
Side                      0
City                      0
County                    0
State                     0
Zipcode                1319
Country                   0
Timezone               3659
Airport_Code           9549
Temperature(F)            0
Humidity(%)               0
Pressure(in)              0
Visibility(mi)            0
Wind_Direction            0
Wind_Speed(mph)           0
Precipitation(in)         0
Weather_Condition         0
Amenity                   0
Bump                      0
Crossing                  0
Give_Way                  0
Junction                  0
No_Exit                   0
Railway                   0
Roundabout                0
Station                   0
Stop                      0
Traffic_Calming           0
Traffic_Signal            0
Turning_Loop        

## Road NaNs
- Remove the rows with 2 streets
- Fill zipcode values with zipcode of nearest accident to it

In [27]:
# remove the 2 streets
df = df[df['Street'].notna()]
df.reset_index(inplace=True)
df.drop('index', axis = 1, inplace=True)

/Users/jonathanwozny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### This function returns a zipcode for the matching city and state, otherwise returns the most occuring in the state
#### It assumes that City, State, Start_Lat, Start_Lng
#### This has high complexity

In [28]:
def FillNearestZipcode(row, col, data):
    closest_zip = 'None'
    closest_ind = -1
    closest_dist = float(sys.maxsize)
    coords_1 = [row['Start_Lat'], row['Start_Lng']]
    
    # search to find nearest accident in the same city
    temp_df = data.loc[(data['State'] == row['State']) & (data['City'] == row['City'])]
    
    if (len(temp_df.index)): # if not empty
        for index, r in temp_df.iterrows():
            coords_2 = [r['Start_Lat'], r['Start_Lng']]
            dist = geopy.distance.geodesic(coords_1, coords_2).mi
            if (dist < closest_dist):
                closest_ind = index
                closest_zip = r[col]
                closest_dist = dist
    
    else: # if empty
        for index, r in data.loc[data['State'] == row['State']].iterrows():
            if index > 10:
                break
            coords_2 = [r['Start_Lat'], r['Start_Lng']]
            dist = geopy.distance.geodesic(coords_1, coords_2).mi
            if (dist < closest_dist):
                closest_ind = index
                closest_zip = r[col]
                closest_dist = dist
    
    row[col] = closest_zip
    return row
    

In [29]:
# fill the nan values for zipcode
notna_zip_df = df.loc[df['Zipcode'].notna(), ['Start_Lat', 'Start_Lng', 'City', 'State', 'Zipcode']]
df.loc[df['Zipcode'].isna()] = df.loc[df['Zipcode'].isna()].apply(lambda x: FillNearestZipcode(x, 'Zipcode', notna_zip_df), axis=1)
# get rid of second dash in zipcode
df['Zipcode'] = df['Zipcode'].str[:5]

/Users/jonathanwozny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-29-86ff39c0724d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Zipcode'] = df['Zipcode'].str[:5]


In [30]:
df.Zipcode.isna().sum()

0

# Create Final Tables

## Create Location Table

In [32]:
#Location(ID_ST : integer, State: string, City : string)
# columns to take from Accident Dataset
Location_cols = ['City', 'State']
temp = df[Location_cols]
Location = temp.copy()

# get unique values
Location.drop_duplicates(inplace = True)

# reset indices
Location.reset_index(inplace=True)
Location.drop('index', axis = 1, inplace=True)

# add ID_ST column (index)
Location.reset_index(inplace=True)
Location.rename(columns={'index':'ID_ST'}, inplace=True)


In [33]:
Location

,ID_ST,City,State
0,0,Dublin,OH
1,1,Dayton,OH
2,2,Cincinnati,OH
3,3,Akron,OH
4,4,Williamsburg,OH
...,...,...,...
16932,16932,Bridgeboro,GA
16933,16933,Buffalo,KS
16934,16934,Stoneham,ME
16935,16935,Plainfield,VT


## Create Environment Table

In [34]:
# Environment(Env_ID : integer, Precipitation : decimal, Visibility : integer, 
#             Temperature : decimal, Humidity : decimal, weather_condition : string, 
#             Light_Or_Darkness : boolean, Air_pressure : decimal, Wind_Speed : decimal, Wind_Direction : string) 
Environment_cols = ['Precipitation(in)', 'Visibility(mi)', 'Temperature(F)', 'Humidity(%)', 'Weather_Condition',
                   'Light_Or_Darkness', 'Pressure(in)', 'Wind_Speed(mph)', 'Wind_Direction']

# create Environment table
temp = df[Environment_cols]
Environment = temp.copy()

# get unique values
Environment.drop_duplicates(inplace = True)

# reset indices
Environment.reset_index(inplace=True)
Environment.drop('index', axis = 1, inplace=True)

# # create Env_ID
Environment.reset_index(inplace=True)
Environment.rename(columns={'index':'Env_ID'}, inplace=True)



In [35]:
Environment.isna().sum()

Env_ID               0
Precipitation(in)    0
Visibility(mi)       0
Temperature(F)       0
Humidity(%)          0
Weather_Condition    0
Light_Or_Darkness    0
Pressure(in)         0
Wind_Speed(mph)      0
Wind_Direction       0
dtype: int64

In [36]:
Environment

,Env_ID,Precipitation(in),Visibility(mi),Temperature(F),Humidity(%),Weather_Condition,Light_Or_Darkness,Pressure(in),Wind_Speed(mph),Wind_Direction
0,0,0.00,10,42.1,58.0,Rain,Night,29.76,0,Calm
1,1,0.02,10,36.9,91.0,Rain,Night,29.68,0,Calm
2,2,0.02,10,36.0,97.0,Clear,Night,29.70,0,Calm
3,3,0.00,10,39.0,55.0,Clear,Night,29.65,0,Calm
4,4,0.01,10,37.0,93.0,Rain,Day,29.69,0,Calm
...,...,...,...,...,...,...,...,...,...,...
1069759,1069759,0.00,10,93.0,18.0,Clear,Day,27.20,0,Calm
1069760,1069760,0.00,8,80.0,48.0,Clear,Day,29.73,0,Calm
1069761,1069761,0.00,10,89.0,34.0,Clear,Day,28.78,0,Calm
1069762,1069762,0.00,10,88.0,32.0,Clear,Day,28.20,0,Calm


## Create Road Table

In [37]:
pd.set_option("display.max_rows", 10)

In [38]:
# Road (Street : string, Zip_Code : integer, Highway : boolean, Obstacle :  string, ID_ST: integer) 
Road_cols = ['Street', 'Zipcode', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 
             'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
            'City', 'State']

# create Road table
temp = df[Road_cols]
Road = temp.copy()

# add highway boolean column to replace last column with values
# all values that would denote a highway
highway_indicators = ['-', 'Hwy', 'hwy' 'Highway', 'highway', 'Expy', 'expy', 'Expressway', 'expressway', 
                    'Fwy', 'fwy', 'Freeway', 'freeway', 'Tollway', 'tollway', 'Tlwy', 'tlwy', 
                    'Tpk', 'tpk', 'Tpke', 'tpke', 'Turnpike', 'turnpike', 'Pkwy', 'pkwy', 'Parkway', 'parkway', 
                    'State', 'state']

Road['Highway'] = 0 # instantiate column
Road.loc[Road['Street'].str.contains('|'.join(highway_indicators)) == True, 'Highway'] = 1

# create Obstacle column with obstacles that are present
Obst_cols = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 
             'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']
Road['Obstacle'] = 'None'

for col in Obst_cols:
    Road.loc[Road[col], 'Obstacle'] = col
    
# remove boolean obstacle columns
Road.drop(Obst_cols, axis = 1, inplace=True)

# get unique values and reset indices
Road.drop_duplicates(inplace = True)

Road.reset_index(inplace=True)
Road.drop('index', axis = 1, inplace=True)

# add ID_ST: merge with Location and get rid of City and State to get ID_ST
merge_cols = ['City', 'State']
Road = pd.merge(Road, Location, how='inner', left_on=merge_cols, right_on=merge_cols).drop(merge_cols, axis = 1)



In [39]:
Road

,Street,Zipcode,Highway,Obstacle,ID_ST
0,Outerbelt E,43017,0,None,0
1,Muirfield Dr,43017,0,None,0
2,Avery Rd,43017,0,None,0
3,Outerbelt N,43016,0,None,0
4,Columbus Marysville Rd S,43017,0,None,0
...,...,...,...,...,...
383905,GA Highway 112 S,31705,0,None,16932
383906,US-75 N,66717,1,Stop,16933
383907,Maine St,04231,0,None,16934
383908,US Route 2,05667,0,None,16935


## Create Airport Table

In [40]:
# Airport(ICAO_ID : string, Name : string, Size : char, Latitude : decimal, Longitude : decimal, ID_ST : integer) 
Airport_cols = ['ident', 'name', 'type', 'latitude_deg', 'longitude_deg', 'municipality', 'local_region']

# create Airport table
temp = df[Airport_cols]
Airport = temp.copy()

# drop first row of metadata
# Airport.drop(index=df.index[0], axis=0, inplace=True)

# get unique values
Airport.drop_duplicates(inplace = True)

# reset indices
Airport.reset_index(inplace=True)
Airport.drop('index', axis = 1, inplace=True)

# add ID_ST: merge with Location and get rid of City and State to get ID_ST
Airport = Airport.merge(Location, left_on=['municipality','local_region'], right_on = ['City','State'], how='inner')



In [41]:
Airport

,ident,name,type,latitude_deg,longitude_deg,municipality,local_region,ID_ST,City,State
0,KOSU,The Ohio State University Airport - Don Scott ...,medium_airport,40.079800,-83.072998,Columbus,OH,10,Columbus,OH
1,KCMH,John Glenn Columbus International Airport,large_airport,39.998001,-82.891899,Columbus,OH,10,Columbus,OH
2,KTZR,Bolton Field,small_airport,39.901199,-83.136902,Columbus,OH,10,Columbus,OH
3,KLCK,Rickenbacker International Airport,medium_airport,39.813801,-82.927803,Columbus,OH,10,Columbus,OH
4,KFFO,Wright-Patterson Air Force Base,medium_airport,39.826099,-84.048302,Dayton,OH,1,Dayton,OH
...,...,...,...,...,...,...,...,...,...,...
3143,KOZA,Ozona Municipal Airport,small_airport,30.735300,-101.203003,Ozona,TX,16878,Ozona,TX
3144,KAVK,Alva Regional Airport,small_airport,36.773201,-98.669899,Alva,OK,16883,Alva,OK
3145,KF08,Eufaula Municipal Airport,small_airport,35.298043,-95.627378,Eufaula,OK,16892,Eufaula,OK
3146,US-5325,Hatch Airport,small_airport,37.649460,-112.418390,Hatch,UT,16925,Hatch,UT


## Create Accident Table

### Should I split date_time into date and time, might be easier to look at stuff

In [42]:
# Accident(ID : integer, Severity : integer, Latitude : decimal, Longitude : decimal, 
#          Distance_of_traffic : decimal, Date_time : datetime, Env_ID : integer, 
#         Street : string, Zip_Code : integer, ICAO_ID : string, Distance : decimal) 
Accident_cols = ['Severity', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Start_Time'] +  Environment_cols + ['Street', 'Zipcode'] + ['ident', 'Nearest_Air_Dist'] 

Accident = df[Accident_cols]

# get Env_ID - merge with environments to get this id
Accident = Accident.merge(Environment, left_on=Environment_cols, right_on = Environment_cols, how='inner').drop(Environment_cols, axis = 1)

# get Street and Zip_Code as foreign keys - ALREADY PART OF TABLE
# Accident = Accident.merge(Road, left_on=['Street', 'Zipcode'], right_on = ['Street', 'Zipcode'], how='inner')


In [43]:
Accident

,Severity,Start_Lat,Start_Lng,Distance(mi),Start_Time,Street,Zipcode,ident,Nearest_Air_Dist,Env_ID
0,3,40.108910,-83.092860,3.230,2016-02-08 00:37:08,Outerbelt E,43017,KOSU,2.267484,0
1,2,39.865420,-84.062800,0.747,2016-02-08 05:56:20,I-70 E,45424,KFFO,2.820266,1
2,2,39.102660,-84.524680,0.055,2016-02-08 06:15:39,I-75 S,45203,KLUK,5.701761,2
3,2,41.062130,-81.537840,0.123,2016-02-08 06:51:45,I-77 N,44311,KAKR,4.076920,3
4,3,39.172393,-84.492792,0.500,2016-02-08 07:53:43,I-75 S,45217,KLUK,6.213225,4
...,...,...,...,...,...,...,...,...,...,...
2845279,2,34.629366,-118.153009,0.000,2019-08-23 15:24:47,Antelope Valley Fwy N,93551,KPMD,3.875102,1069759
2845280,2,34.072330,-118.080660,0.165,2019-08-23 15:23:31,I-10 W,91770,KEMT,2.785205,1069760
2845281,2,34.030470,-117.598170,0.548,2019-08-23 16:26:06,Pomona Fwy E,91761,KONT,1.767090,1069761
2845282,2,33.943599,-117.077880,0.000,2019-08-23 17:40:12,CA-60 E,92555,KRIV,11.274882,1069762


In [45]:
# change column names to column names in database
#### Location
# NOT NEEDED

#### Environment
env_rename = {'Precipitation(in)':'Precipitation' , 'Visibility(mi)' : 'Visibility', 'Temperature(F)' : 'Temperature', 
              'Humidity(%)' : 'Humidity', 'Pressure(in)' : 'Pressure', 'Wind_Speed(mph)' : 'Wind_Speed'}
Environment.rename(columns = env_rename, inplace=True)

#### Road
road_rename = {'Zipcode':'Zip_Code'}
Road.rename(columns = road_rename, inplace=True)

#### Airport
# Airport(ICAO_ID : string, Name : string, Size : char, Latitude : decimal, Longitude : decimal, ID_ST : integer) 
air_rename = {'ident' : 'ICAO_ID ', 'name' : 'Name', 'type': 'Size', 'latitude_deg' : 'Latitude', 'longitude_deg' : 'Longitude'}
Airport.rename(columns = air_rename, inplace=True)

#### Accident
acc_rename = {'Start_Lat' : 'Latitude', 'Start_Lng' : 'Longitude', 'Distance(mi)' : 'Distance_of_traffice', 
              'Start_Time' : 'Date_time', 'ident' : 'ICAO_ID'}


In [46]:
# send to csv files
Accident.to_csv('Accident.csv', index = False)
Airport.to_csv('Airport.csv', index = False)
Road.to_csv('Road.csv', index = False)
Environment.to_csv('Environment.csv', index = False)
Location.to_csv('Location.csv', index = False)